In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
DIRECTORY = '/content/drive/My Drive/Informatics/Sphere@mail.ru/1_Data_Analysis/Lec_11/'

# Введение в SQL

<img width = '400px' src="images/lesson_11/tds1qm.jpg">

**Реляционные vs нерреляционные БД**

**Реляционная база данных** – это набор данных с предопределенными связями между ними. Эти данные организованны в виде набора таблиц, состоящих из столбцов и строк. 

**Реляционные БД** хранят структурированные данные, которые обычно представляют объекты реального мира. Скажем, это могут быть сведения о человеке, или о содержимом корзины для товаров в магазине, сгруппированные в таблицах, формат которых задан на этапе проектирования хранилища.

**Нереляционные БД** предлагают динамическую структуру данных, которые могут храниться несколькими способами: ориентированно по колонкам, документо-ориентированно, в виде графов или на основе пар «ключ-значение».

<img width = '600px' src="images/lesson_11/dbs.jpg">

**Язык SQL**

**SQL** (ˈɛsˈkjuˈɛl; англ. structured query language — «язык структурированных запросов») — декларативный язык программирования, применяемый для создания, модификации и управления данными в реляционной базе данных.

SQL используется для добавления, обновления и удаления строк данных, извлечения наборов данных для обработки транзакций и аналитических приложений, а также для управления всеми аспектами работы базы данных.

In [ ]:
import sqlite3
import pandas as pd

СУБД SQLite не обращает внимания на типы данных (она хранит все данные в виде
строк).

Схема работы с БД:
- Подключение к базе данных (вызов connect() с получением объекта - соединения).
- Создание одного или нескольких курсоров (вызов метода объекта-соединения cursor() - CURrrent Set Of Records, текущий набор записей).
- Исполнение команды или запроса (вызов метода execute() или его
вариантов).
- Получение результатов запроса (вызов метода fetchone() или его
вариантов).
- Завершение транзакции или ее откат (вызов метода объекта-соединения
commit() или rollback() ).
- Когда все необходимые транзакции произведены, подключение
закрывается вызовом метода close() объекта-соединения.


In [ ]:
import sqlite3
connection = sqlite3.connect("company.db")

cursor = connection.cursor()

# delete 
# cursor.execute("""DROP TABLE employee;""")

sql_command = """
CREATE TABLE employee 
(  
    staff_number INTEGER PRIMARY KEY, 
    fname VARCHAR(20), 
    lname VARCHAR(30) DEFAULT 'Ivanov', 
    gender CHAR(1), 
    joining DATE,
    birth_date DATE
);"""

cursor.execute(sql_command)

sql_command = """INSERT INTO employee (staff_number, fname, lname, gender, birth_date)
    VALUES (1324, "William", "Shakespeare", "m", "1961-10-25");"""
cursor.execute(sql_command)


sql_command = """UPDATE employee SET fname = 'Ivan' WHERE gender = 'm' """
cursor.execute(sql_command)

sql_command = """DELETE FROM employee WHERE lname = 'Petrov' """
cursor.execute(sql_command)

connection.commit()

connection.close()

In [ ]:
connection = sqlite3.connect("company.db")

cursor = connection.cursor()

staff_data = [ ("William", "Shakespeare", "m", "1961-10-25"),
               ("Frank", "Schiller", "m", "1955-08-17"),
               ("Jane", "Wall", "f", "1989-03-14") ]
               
for p in staff_data:
    format_str = """INSERT INTO employee (staff_number, fname, lname, gender, birth_date)
    VALUES (NULL, "{first}", "{last}", "{gender}", "{birthdate}");"""

    sql_command = format_str.format(first=p[0], last=p[1], gender=p[2], birthdate = p[3])
    cursor.execute(sql_command)

connection.commit()

connection.close()    

### CREATE TABLE

Создание базовой таблицы.

CREATE TABLE <имя таблицы>(<список спецификаций столбцов и ограничений>);

In [ ]:
sql_command = """
CREATE TABLE employee ( 
                        staff_number INTEGER PRIMARY KEY, 
                        fname VARCHAR(20), 
                        lname VARCHAR(30) DEFAULT 'Ivanov', 
                        gender CHAR(1), 
                        joining DATE,
                        birth_date DATE                    
                       );"""

Для обеспечения категорной целостности в языке SQL существуют спецификации **PRIMARY KEY** (первичный ключ) и **UNIQUE** (уникальный ключ). 

 - PRIMARY KEY не может быть NULL
 - PRIMARY KEY может быть только один

### INSERT

Вставка новых записей в таблицу.

INSERT INTO <имя таблицы>[(<имя столбца>,...)]

{VALUES (<значение столбца>,…)}

        | <выражение запроса>     
    
        | {DEFAULT VALUES}

Если отсутствует список столбцов, то список вставляемых значений должен быть полный, то есть обеспечивать значения для всех столбцов таблицы. При этом порядок значений должен соответствовать порядку, заданному оператором CREATE TABLE для таблицы, в которую вставляются строки. 

In [ ]:
sql_command = """INSERT INTO employee (staff_number, fname, lname, gender, birth_date)
    VALUES (1324, "William", "Shakespeare", "m", "1961-10-25");"""

In [ ]:
sql_command = """INSERT INTO employee
    VALUES (1324, "William", "Shakespeare", "m", "1961-10-25", NULL);"""

In [ ]:
sql_command = """INSERT INTO employee
    VALUES (1324, "William", DEFAULT, "m", "1961-10-25", NULL);"""

### UPDATE

Изменяет имеющиеся данные в таблице.

UPDATE <имя таблицы>

SET {<имя столбца> = {<выражение для вычисления значения столбца>
                      
        | NULL
                      
        | DEFAULT},...}

        [ {WHERE <предикат>}]

In [ ]:
sql_command = """UPDATE employee SET fname = 'Ivan' WHERE gender = 'm' """

### DELETE

Удаляет строки.

DELETE FROM <имя таблицы > 

[WHERE <предикат>];

In [ ]:
sql_command = """DELETE FROM employee WHERE lname = 'Petrov' """

### База данных computers

Взято с ресурса http://www.sql-tutorial.ru/.

<img width = '500px' src="images/lesson_11/computers.gif">

In [ ]:
def sql_query(SQL_QUERY, db):
    connection = sqlite3.connect(db)
    cursor = connection.cursor()
    try:
        if str.lower(SQL_QUERY)[:6].strip() == 'select':
            df = pd.read_sql_query(SQL_QUERY, connection)
            return df

        sql_command = SQL_QUERY
        cursor.execute(sql_command)
    finally:
        connection.commit()
        connection.close()

### SELECT

Оператор SELECT осуществляет выборку из базы данных.

In [ ]:
sql_query("""SELECT * FROM product """, 'computers.db')

,maker,model,type
0,B,1121,PC
1,A,1232,PC
2,A,1233,PC
3,E,1260,PC
4,A,1276,Printer
5,D,1288,Printer
6,A,1298,Laptop
7,C,1321,Laptop
8,A,1401,Printer
9,A,1408,Printer


In [ ]:
sql_query("""SELECT model, code FROM pc """, 'computers.db')

,model,code
0,1232,1
1,1121,2
2,1233,3
3,1121,4
4,1121,5
5,1233,6
6,1232,7
7,1232,8
8,1232,9
9,1260,10


### Упражнение

Добавить в таблицу Product следующие продукты производителя Z: 
принтер модели 4003, ПК модели 4001 и блокнот модели 4002. 

In [ ]:
# Выполните SELECT из БД, убедитесь, что все сделали верно.
sql_query("""SELECT * FROM product """, 'computers.db')

,maker,model,type
0,B,1121,PC
1,A,1232,PC
2,A,1233,PC
3,E,1260,PC
4,A,1276,Printer
5,D,1288,Printer
6,A,1298,Laptop
7,C,1321,Laptop
8,A,1401,Printer
9,A,1408,Printer


**DISTINCT**  - только уникальные значения.

In [ ]:
sql_query("""SELECT distinct model FROM pc """, 'computers.db')

,model
0,1232
1,1121
2,1233
3,1260


**ORDER BY** -сортировка запроса по полю.

In [ ]:
sql_query("""SELECT distinct model FROM pc ORDER BY 1""", 'computers.db')

,model
0,1121
1,1232
2,1233
3,1260


In [ ]:
sql_query("""SELECT distinct model FROM pc ORDER BY model DESC""", 'computers.db')

,model
0,1260
1,1233
2,1232
3,1121


**WHERE** - задает условие на горизонтальную выборку.

In [ ]:
sql_query("""SELECT * FROM pc WHERE model = 1260 """, 'computers.db')

,code,model,speed,ram,hd,cd,price
0,10,1260,500,32,10.0,12x,350


<img width = '500px' src="images/lesson_11/imgs_.png">

Переименование столбцов:

In [ ]:
sql_query("""SELECT ram AS Mb, hd Gb FROM PC WHERE cd = '24x'; """, 'computers.db')

,Mb,Gb
0,64,8.0
1,32,10.0


### Предикаты SQL

**AND, OR, NOT**

In [ ]:
# модели, не являющиеся ПК или принтером, т.е. модели ноутбуков в нашем случае
sql_query("""SELECT * FROM Product WHERE NOT (type='PC' OR type='Printer'); """, 'computers.db')

,maker,model,type
0,A,1298,Laptop
1,C,1321,Laptop
2,B,1750,Laptop
3,A,1752,Laptop
4,Z,4002,Laptop


**EXISTS/NOT EXISTS**

Предикат EXISTS принимает значение TRUE, если подзапрос содержит любое количество строк, иначе его значение равно FALSE. 

In [ ]:
# Найти тех производителей портативных компьютеров, которые также производят принтеры
sql_query("""SELECT DISTINCT maker
            FROM Product AS lap_product
            WHERE type = 'Laptop' AND 
                        EXISTS (
                        SELECT maker
                        FROM Product
                        WHERE type = 'Printer' AND 
                        maker = lap_product.maker
                 );
 
""", 'computers.db')

,maker
0,A
1,Z


**BETWEEN**

In [ ]:
sql_query("""SELECT model, speed FROM PC WHERE price BETWEEN 400 AND 600;""", 'computers.db')

,model,speed
0,1232,500
1,1233,500
2,1232,500


**IN**

In [ ]:
sql_query("""SELECT model, speed, hd FROM PC WHERE hd IN (10, 20);""", 'computers.db')

,model,speed,hd
0,1233,750,20.0
1,1232,500,10.0
2,1232,450,10.0
3,1260,500,10.0
4,1233,800,20.0


### Упражнение

Найдите номер модели, объем памяти и размеры экранов ПК-блокнотов, цена которых превышает 1000 дол.

Найдите номер модели, скорость и размер жесткого диска ПК, имеющих 12x или 24x CD и цену менее 600 дол.

**LIKE**

Символ (_) -  вместо любого единичного символа в проверяемом значении;

Символ (%) - заменяет последовательность любых символов (число символов в последовательности может быть от 0 и более) в проверяемом значении.

<img width = '700px' src="images/lesson_11/imgs_2.png">

In [ ]:
sql_query("""SELECT * FROM product WHERE type LIKE 'P%'  """, 'computers.db')

,maker,model,type
0,B,1121,PC
1,A,1232,PC
2,A,1233,PC
3,E,1260,PC
4,A,1276,Printer
5,D,1288,Printer
6,A,1401,Printer
7,A,1408,Printer
8,D,1433,Printer
9,E,1434,Printer


In [ ]:
sql_query("""SELECT * FROM product WHERE type LIKE 'P%' and type not like 'PC%'  """, 'computers.db')

,maker,model,type
0,A,1276,Printer
1,D,1288,Printer
2,A,1401,Printer
3,A,1408,Printer
4,D,1433,Printer
5,E,1434,Printer
6,Z,4003,Printer


In [ ]:
sql_query("""SELECT * FROM product WHERE model LIKE '12%' """, 'computers.db')

,maker,model,type
0,A,1232,PC
1,A,1233,PC
2,E,1260,PC
3,A,1276,Printer
4,D,1288,Printer
5,A,1298,Laptop


### Упражнение

Найди все модели принтеров, которые начинаюстя на 1 и заканчиваются на 1.

### Агрегаты

**SUM, MIN, MAX, AVG, COUNT** 

In [ ]:
sql_query("""SELECT MIN(price) AS Min_price, MAX(price) AS Max_price, count(price) FROM PC; """, 'computers.db')

,Min_price,Max_price,count(price)
0,350,980,12


### Упражнение

Найти имеющееся в наличии количество компьютеров, выпущенных производителем А.

Найти количество имеющихся различных моделей ПК, выпускаемых производителем А.

**GROUP BY/HAVING**

<img width = '700px' src="images/lesson_11/sql-having.png">

**GROUP BY** используется для объединения результатов выборки по одному или нескольким столбцам.

In [ ]:
sql_query("""SELECT model, COUNT(model) AS Qty_model, 
    AVG(price) AS Avg_price
FROM PC
GROUP BY model; """, 'computers.db')

,model,Qty_model,Avg_price
0,1121,3,850.0
1,1232,4,425.0
2,1233,4,875.0
3,1260,1,350.0


Если предложение WHERE определяет предикат для фильтрации строк, то предложение **HAVING** применяется после группировки для определения аналогичного предиката, фильтрующего группы по значениям агрегатных функций. 

In [ ]:
sql_query("""SELECT model, COUNT(model) AS Qty_model, 
    AVG(price) AS Avg_price
FROM PC
GROUP BY model
HAVING avg(price) > 500
; """, 'computers.db')

,model,Qty_model,Avg_price
0,1121,3,850.0
1,1233,4,875.0


Правильный порядок:
    
**SELECT** <выражение>

**FROM** <имя таблицы> 

[**WHERE** <предикат>]

[[**GROUP BY** <список столбцов>]

[**HAVING** <условие на агрегатные значения>] ]

[**ORDER BY** <список столбцов>]

### Упражнение

Найдите производителей, выпускающих по меньшей мере три различных модели ПК. Вывести: Maker, число моделей ПК.

Найдите средний размер диска ПК (одно значение для всех) тех производителей, которые выпускают и принтеры. Вывести: средний размер HD

### Выбор из нескольких источников.

In [ ]:
sql_query("""SELECT *
FROM product
where type = 'Printer'
 """, 'computers.db')

,maker,model,type
0,A,1276,Printer
1,D,1288,Printer
2,A,1401,Printer
3,A,1408,Printer
4,D,1433,Printer
5,E,1434,Printer
6,Z,4003,Printer


In [ ]:
#Декартово произведение
sql_query("""SELECT *
FROM product, (SELECT distinct type as type_of_printer FROM printer)
where type = 'Printer'
 """, 'computers.db')

,maker,model,type,type_of_printer
0,A,1276,Printer,Laser
1,A,1401,Printer,Laser
2,A,1408,Printer,Laser
3,D,1288,Printer,Laser
4,D,1433,Printer,Laser
5,E,1434,Printer,Laser
6,Z,4003,Printer,Laser
7,A,1276,Printer,Jet
8,A,1401,Printer,Jet
9,A,1408,Printer,Jet


<img width = '700px' src="images/lesson_11/joins.png">

Найти производителя, номер модели и цену каждого компьютера, имеющегося в базе данных:

In [ ]:
sql_query("""SELECT maker, Product.model AS model_1, 
            PC.model AS model_2, price
FROM Product INNER JOIN 
 PC ON PC.model = Product.model
ORDER BY maker, model_2;
 """, 'computers.db')

,maker,model_1,model_2,price
0,A,1232,1232,350
1,A,1232,1232,350
2,A,1232,1232,400
3,A,1232,1232,600
4,A,1233,1233,600
5,A,1233,1233,950
6,A,1233,1233,970
7,A,1233,1233,980
8,B,1121,1121,850
9,B,1121,1121,850


In [ ]:
sql_query("""SELECT maker, Product.model AS model_1, 
            PC.model AS model_2, price
FROM Product LEFT JOIN 
 PC ON PC.model = Product.model
ORDER BY maker, model_2;
 """, 'computers.db')

,maker,model_1,model_2,price
0,A,1276,None,NaN
1,A,1298,None,NaN
2,A,1401,None,NaN
3,A,1408,None,NaN
4,A,1752,None,NaN
5,A,1232,1232,350.0
6,A,1232,1232,350.0
7,A,1232,1232,400.0
8,A,1232,1232,600.0
9,A,1233,1233,600.0


### Упражнение

Найдите производителей, которые производили бы как ПК со скоростью не менее 750 МГц, так и ПК-блокноты со скоростью не менее 750 МГц. 

**UNION**

Предложение **UNION** приводит к появлению в результирующем наборе всех строк каждого из запросов. При этом, если определен параметр **ALL**, то сохраняются все дубликаты выходных строк, в противном случае в результирующем наборе присутствуют только уникальные строки.

Операция объединения может быть выполнена только при выполнении следующих условий:

- количество выходных столбцов каждого из запросов должно быть одинаковым;

- выходные столбцы каждого из запросов должны быть совместимы между собой (в порядке их следования) по типам данных;

- в результирующем наборе используются имена столбцов, заданные в первом запросе;

Найти номера моделей и цены ПК и портативных компьютеров:

In [ ]:
sql_query("""SELECT model, price
FROM PC
UNION
SELECT model, price
FROM Laptop
ORDER BY price DESC;
 """, 'computers.db')

,model,price
0,1750,1200
1,1752,1150
2,1298,1050
3,1233,980
4,1233,970
5,1321,970
6,1233,950
7,1298,950
8,1121,850
9,1298,700


### Упражнение

Найти тип продукции, номер модели и цену ПК и портативных компьютеров:

**UNION ALL**

Найти все имеющиеся единицы продукции производителя 'B'. Вывести номер модели и тип.

In [ ]:
sql_query("""SELECT p.model, p.type FROM pc JOIN Product p ON PC.model=p.model WHERE maker='B'
UNION ALL
SELECT p.model, p.type FROM printer pr JOIN Product p ON pr.model=p.model WHERE maker='B'
UNION ALL
SELECT p.model, p.type FROM laptop lp JOIN Product p ON lp.model=p.model WHERE maker='B';
 """, 'computers.db')

,model,type
0,1121,PC
1,1121,PC
2,1121,PC
3,1750,Laptop


In [ ]:
sql_query("""SELECT p.model, p.type FROM pc JOIN Product p ON PC.model=p.model WHERE maker='B'
UNION 
SELECT p.model, p.type FROM printer pr JOIN Product p ON pr.model=p.model WHERE maker='B'
UNION
SELECT p.model, p.type FROM laptop lp JOIN Product p ON lp.model=p.model WHERE maker='B';
 """, 'computers.db')

,model,type
0,1121,PC
1,1750,Laptop


### Упражнение

Найдите среднюю цену ПК и ПК-блокнотов, выпущенных производителем A (латинская буква). Вывести: одна общая средняя цена.